<a href="https://colab.research.google.com/github/Dhruv-Solutions/Brain_Tumor_Classification/blob/main/Brain_Tumor_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Linux distro and kernel
!cat /etc/issue && uname -a

# CPU info
!lscpu

# Memory
!free -h

# (If GPU enabled)
!nvidia-smi


Ubuntu 22.04.4 LTS \n \l

Linux fcfe3aff0523 6.1.123+ #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025 x86_64 x86_64 x86_64 GNU/Linux
Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.33
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht sysc
                          all nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xt
                          opology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq
        

In [1]:
import tensorflow as tf
print(tf.__version__)   # e.g. "2.12.0"


2.18.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
from collections import Counter
from PIL import Image

BASE = "/content/drive/MyDrive/Colab Notebooks/Tumour"
splits = ["train", "valid", "test"]
for split in splits:
    path = os.path.join(BASE, split)
    counts = {
    d: len([f for f in os.listdir(os.path.join(path,d))
            if os.path.isfile(os.path.join(path,d,f))
               and f.lower().endswith(('.png','.jpg','.jpeg'))])
    for d in os.listdir(path)
    if os.path.isdir(os.path.join(path,d))
}
print(f"{split.upper():6}", counts)




TEST   {'No Tumor': 49, 'glioma': 80, 'pituitary': 54, 'meningioma': 63}


In [5]:
# Data Augmentation
import tensorflow as tf
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1,0.1),
])
print(tf.__version__)

2.18.0


In [6]:
from tensorflow.keras.layers import Rescaling


rescale = Rescaling(1./255)
def make_dataset(split_dir, batch=32, augment=False):
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        split_dir, image_size=(224, 224), batch_size=batch,
        label_mode='categorical', shuffle=(split_dir.endswith("train"))
    )
    ds = ds.map(lambda x,y: (rescale(x), y),
                num_parallel_calls=tf.data.AUTOTUNE)
    # … (apply augmentation here if requested)
    if augment:
        ds = ds.map(lambda x,y: (data_augmentation(x, training=True), y),
                    num_parallel_calls=tf.data.AUTOTUNE)
    return ds.prefetch(tf.data.AUTOTUNE)

In [7]:
from tensorflow.keras import layers, models

def build_custom_cnn(input_shape=(224,224,3), num_classes=4):
    model = models.Sequential([
        layers.Input(input_shape),
        layers.Conv2D(32,3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        layers.Conv2D(64,3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        layers.Conv2D(128,3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from preprocessing import make_dataset
#from model_custom import build_custom_cnn

train_ds = make_dataset(os.path.join(BASE,"train"), batch=32, augment=True)
val_ds   = make_dataset(os.path.join(BASE,"valid"), batch=32)

model = build_custom_cnn()
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("models/custom_cnn.keras", save_best_only=True)
]
history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=callbacks)


Found 1695 files belonging to 4 classes.
Found 502 files belonging to 4 classes.
Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 525s 10s/step - accuracy: 0.5924 - loss: 1.3853 - val_accuracy: 0.2470 - val_loss: 4.3751
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 27s 475ms/step - accuracy: 0.7087 - loss: 0.8423 - val_accuracy: 0.2470 - val_loss: 9.3586
Epoch 3/50
 3/53 ━━━━━━━━━━━━━━━━━━━━ 32s 653ms/step - accuracy: 0.7830 - loss: 0.5846

In [ ]:
## Model Transfer
import tensorflow as tf
from tensorflow.keras import layers, models

def build_transfer_model(base_name='ResNet50', input_shape=(224,224,3), num_classes=4):
    base = getattr(tf.keras.applications, base_name)(
        include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg'
    )
    base.trainable = False

    x = layers.Input(shape=input_shape)
    y = base(x, training=False)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.5)(y)
    output = layers.Dense(num_classes, activation='softmax')(y)

    model = models.Model(x, output)
    model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

    return model


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from preprocessing import make_dataset
#from model_transfer import build_transfer_model

train_ds = make_dataset(os.path.join(BASE,"train"), batch=32, augment=True)
val_ds   = make_dataset(os.path.join(BASE,"valid"), batch=32)

model = build_transfer_model('EfficientNetB0')
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint("models/transfer_efficientnet.keras", save_best_only=True)
]
model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=callbacks)


Found 1695 files belonging to 4 classes.
Found 502 files belonging to 4 classes.
Epoch 1/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 16s 237ms/step - accuracy: 0.6302 - loss: 1.1127 - val_accuracy: 0.8526 - val_loss: 0.3899
Epoch 2/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.8063 - loss: 0.5162 - val_accuracy: 0.8247 - val_loss: 0.4436
Epoch 3/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.8621 - loss: 0.3595 - val_accuracy: 0.8765 - val_loss: 0.3639
Epoch 4/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 189ms/step - accuracy: 0.8846 - loss: 0.3308 - val_accuracy: 0.8944 - val_loss: 0.2867
Epoch 5/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 185ms/step - accuracy: 0.8914 - loss: 0.2945 - val_accuracy: 0.9024 - val_loss: 0.2898
Epoch 6/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.9118 - loss: 0.2672 - val_accuracy: 0.9084 - val_loss: 0.2609
Epoch 7/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.9046 - loss: 0.2624 - val_accuracy: 0.9104 - val_loss: 0.2319
Epoch 8/40
53/53

In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 1280)           │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,544,049 (17.33 MB)

 Trainable params: 164,740 (643.52 KB)

 Non-trainable params: 4,049,827 (15.45 MB)

 Optimizer params: 329,482 (1.26 MB)

In [ ]:
#Training and monitoring
import matplotlib.pyplot as plt

def plot_history(hist, name):
    plt.plot(hist.history['loss'], label='train_loss')
    plt.plot(hist.history['val_loss'], label='val_loss')
    plt.title(name)
    plt.legend()
    plt.show()


In [ ]:
#Evaluation
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
#from preprocessing import make_dataset

test_ds = make_dataset(os.path.join(BASE,"test"), batch=32, augment=False)
model = load_model("models/transfer_efficientnet.keras")
y_true = np.concatenate([y for _,y in test_ds], axis=0)
y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_true, axis=1)

print(classification_report(y_true_labels, y_pred_labels, target_names=test_ds.class_names))
print(confusion_matrix(y_true_labels, y_pred_labels))


Found 246 files belonging to 4 classes.


2025-07-24 20:50:42.415947: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 323ms/step
              precision    recall  f1-score   support

    No Tumor       0.15      0.14      0.14        49
      glioma       0.35      0.39      0.37        80
  meningioma       0.24      0.21      0.22        63
   pituitary       0.24      0.24      0.24        54

    accuracy                           0.26       246
   macro avg       0.24      0.24      0.24       246
weighted avg       0.26      0.26      0.26       246

[[ 7 18 13 11]
 [10 31 16 23]
 [20 22 13  8]
 [11 17 13 13]]


In [ ]:
#Evaluation by Custom CNN
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
#from preprocessing import make_dataset

test_ds = make_dataset(os.path.join(BASE,"test"), batch=32, augment=False)
model = load_model("models/custom_cnn.keras")
y_true = np.concatenate([y for _,y in test_ds], axis=0)
y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_true, axis=1)

print(classification_report(y_true_labels, y_pred_labels, target_names=test_ds.class_names))
print(confusion_matrix(y_true_labels, y_pred_labels))


Found 246 files belonging to 4 classes.
1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step

2025-07-24 20:50:46.663090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
              precision    recall  f1-score   support

    No Tumor       0.15      0.14      0.14        49
      glioma       0.37      0.38      0.37        80
  meningioma       0.27      0.27      0.27        63
   pituitary       0.33      0.33      0.33        54

    accuracy                           0.29       246
   macro avg       0.28      0.28      0.28       246
weighted avg       0.29      0.29      0.29       246

[[ 7 19 12 11]
 [15 30 19 16]
 [17 19 17 10]
 [ 9 13 14 18]]


In [ ]:
import streamlit as st
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

CLASS_NAMES = ["glioma","meningioma","no_tumor","pituitary"]
model = load_model("models/transfer_efficientnet.keras")

st.title("🧠 Brain Tumor MRI Classifier")
uploaded = st.file_uploader("Upload an MRI image", type=["png","jpg","jpeg"])
if uploaded:
    img = Image.open(uploaded).convert("RGB").resize((224,224))
    x = np.array(img)/255.0
    preds = model.predict(x[np.newaxis,...])[0]
    idx = np.argmax(preds)
    st.write(f"**Prediction:** {CLASS_NAMES[idx]} ({preds[idx]*100:.1f}% confidence)")
    st.bar_chart(preds, use_container_width=True)



2025-07-24 20:53:04.571 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.614 
  command:

    streamlit run /Users/dhruvsharma/PyCharmMiscProject/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-24 20:53:04.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:0

In [ ]:
streamlit run /Users/dhruvsharma/PyCharmMiscProject/.venv/lib/python3.9/site-packages/ipykernel_launcher.py

SyntaxError: invalid syntax (2903271827.py, line 1)